### Init

In [1]:
from google.cloud.bigquery import *

In [2]:
import pandas as pd

In [3]:
from utils.bqutils import connect, init_catalog

In [4]:
from bokeh_frame import column

In [5]:
client = connect('bqresearch')

In [6]:
catalog = init_catalog()

## Population

In [8]:
population_per_year = client.query(f'''
select year, population
from `coviddatail.staging.population_table`
where safe_cast(age as NUMERIC) between 40 and 44
and sex='total'
and county='total'
and area='total'
order by year
'''
).to_dataframe()

In [9]:
population_per_year.year = population_per_year.year.astype(float)
population_per_year.population = population_per_year.population.astype(float)
population_per_year = population_per_year.groupby('year').sum().reset_index()

In [10]:
from bokeh_frame import charts

In [ ]:
population_per_year

In [11]:
import numpy as np

### Interpolate and extrapolate population per year

In [210]:
population_per_year_intrp_extrp = population_per_year.copy()

In [211]:
population_per_year_intrp_extrp['month'] = 12
population_per_year_intrp_extrp['day'] = 31
population_per_year_intrp_extrp['date'] = pd.to_datetime(population_per_year_intrp_extrp.loc[:,['year', 'month', 'day']])

In [212]:
population_per_year_intrp_extrp = population_per_year_intrp_extrp.set_index('date').reindex(pd.date_range('2000-12-31', '2022-12-31'))
population_per_year_intrp_extrp['date'] = population_per_year_intrp_extrp.index
population_per_year_intrp_extrp['month'] = population_per_year_intrp_extrp['date'].dt.month
population_per_year_intrp_extrp['day'] = population_per_year_intrp_extrp['date'].dt.day
population_per_year_intrp_extrp['year'] = population_per_year_intrp_extrp['date'].dt.year

In [238]:
L = np.where(~population_per_year_intrp_extrp.population.isna())[0]

In [240]:
l = len(population_per_year_intrp_extrp)
population_per_year_intrp_extrp.population.values[-1]  = \
     population_per_year_intrp_extrp.population.values[L[-1]] + \
        (population_per_year_intrp_extrp.population.values[L[-2]] - 
         population_per_year_intrp_extrp.population.values[L[-3]])* (l-L[-1])/(L[-2]-L[-3])

In [259]:
(population_per_year_intrp_extrp.population.values[L[-2]] - 
         population_per_year_intrp_extrp.population.values[L[-3]])

6162.0

In [247]:
(l-1-L[-1])/(L[-2]-L[-3])

3.0027397260273974

In [245]:
(population_per_year_intrp_extrp.population.values[l-1],
 population_per_year_intrp_extrp.population.values[L[-1]],
 population_per_year_intrp_extrp.population.values[L[-2]],
 population_per_year_intrp_extrp.population.values[L[-3]])
 

(581628.7643835617, 563109.0, 556290.0, 550128.0)

In [233]:
np.where(~population_per_year_intrp_extrp.population.isna())

(array([   0,  365,  730, 1095, 1461, 1826, 2191, 2556, 2922, 3287, 3652,
        4017, 4383, 4748, 5113, 5478, 5844, 6209, 6574, 6939]),)

In [215]:
(population_per_year_intrp_extrp.population.values[L[-1]],
 population_per_year_intrp_extrp.population.values[L[-2]],
 population_per_year_intrp_extrp.population.values[8035])
 

(nan, nan, nan)

In [260]:
population_per_year_intrp_extrp.population = population_per_year_intrp_extrp.population.interpolate()

In [177]:
population_per_year_intrp_extrp.set_index('date').index

DatetimeIndex(['2000-12-31', '2001-12-31', '2002-12-31', '2003-12-31',
               '2004-12-31', '2005-12-31', '2006-12-31', '2007-12-31',
               '2008-12-31', '2009-12-31', '2010-12-31', '2011-12-31',
               '2012-12-31', '2013-12-31', '2014-12-31', '2015-12-31',
               '2016-12-31', '2017-12-31', '2018-12-31', '2019-12-31',
                      'NaT',        'NaT'],
              dtype='datetime64[ns]', name='date', freq=None)

In [43]:
population_per_year_intrp_extrp = population_per_year_intrp_extrp.loc[:,['year', 'population']] = \
    population_per_year_intrp_extrp.loc[:,['year', 'population']].interpolate(method='linear')

In [44]:
population_per_year_intrp_extrp.index.name = 'date'

In [102]:
(population_per_year_intrp_extrp.loc['2019-12-31', 'population'],
population_per_year_intrp_extrp.loc['2020-12-31', 'population'],
population_per_year_intrp_extrp.loc['2021-12-31', 'population'],
population_per_year_intrp_extrp.loc['2022-12-31', 'population']
)



(563109.0, nan, nan, nan)

In [261]:
charts.Line('date', 'population', population_per_year_intrp_extrp).datetime().opts(width=1000)

{'year': '@year', 'population': '@population', 'month': '@month', 'day': '@day', 'date': '@date{%F}'}


Figure(id='4014', ...)

## Deaths

In [28]:
deaths_per_day = client.query('''
WITH total_deaths_per_date as
(select DATE(day) report_date, age_40_44 as total
from staging.deaths_report_1
order by report_date

)
select * from total_deaths_per_date
'''
).to_dataframe()

In [94]:
### Z Score per day of year

In [34]:
deaths_per_day['total'] = deaths_per_day.total.astype(int)

In [35]:
deaths_per_day['date'] = pd.to_datetime(deaths_per_day.report_date)
deaths_per_day.set_index('date', drop=False, inplace=True)

In [36]:
deaths_per_week = deaths_per_day.resample('w').total.sum()

In [38]:
deaths_per_week

date
2000-01-02     6
2000-01-09    15
2000-01-16     9
2000-01-23    12
2000-01-30    11
              ..
2021-11-07     4
2021-11-14    10
2021-11-21     8
2021-11-28     5
2021-12-05     7
Freq: W-SUN, Name: total, Length: 1145, dtype: int64

### normalize deaths by population

In [277]:
deaths_normalized = pd.merge(deaths_per_week, population_per_year_intrp_extrp, left_index=True, right_index=True)

In [279]:
deaths_normalized.index.name='date'

In [280]:
deaths_normalized.drop('date', axis=1, inplace=True)

In [281]:
deaths_normalized['death_per_1000'] = deaths_normalized['total'] / deaths_normalized['population'] * 1000

In [282]:
charts.Line('date', 'death_per_1000', deaths_normalized).datetime().opts(width=1000)

{'total': '@total', 'year': '@year', 'population': '@population', 'month': '@month', 'day': '@day', 'death_per_1000': '@death_per_1000', 'date': '@date{%F}'}


Figure(id='5334', ...)

In [60]:
deaths_normalized.date

AttributeError: 'DataFrame' object has no attribute 'date'

In [59]:
deaths_normalized

,total,year,population,death_per_10k,death_per_1000
date,,,,,
2000-12-31,12,2000,378095.000000,0.317381,0.031738
2001-01-07,8,2001,378156.197260,0.211553,0.021155
2001-01-14,13,2001,378217.394521,0.343718,0.034372
2001-01-21,14,2001,378278.591781,0.370098,0.037010
2001-01-28,7,2001,378339.789041,0.185019,0.018502
...,...,...,...,...,...
2021-11-07,4,2021,563109.000000,0.071034,0.007103
2021-11-14,10,2021,563109.000000,0.177586,0.017759
2021-11-21,8,2021,563109.000000,0.142068,0.014207


In [180]:
deaths_normalized.set_index('date').loc['2000':'2018']

,report_date,total,year,population,month,day,death_per_1000,death_per_10k
date,,,,,,,,
2000-12-31,2000-12-31,4,2000,378095.000000,12,31,0.010579,0.105794
2001-01-01,2001-01-01,1,2001,378103.742466,1,1,0.002645,0.026448
2001-01-02,2001-01-02,1,2001,378112.484932,1,2,0.002645,0.026447
2001-01-03,2001-01-03,1,2001,378121.227397,1,3,0.002645,0.026447
2001-01-04,2001-01-04,1,2001,378129.969863,1,4,0.002645,0.026446
...,...,...,...,...,...,...,...,...
2018-12-27,2018-12-27,0,2018,556222.471233,12,27,0.000000,0.000000
2018-12-28,2018-12-28,3,2018,556239.353425,12,28,0.005393,0.053934
2018-12-29,2018-12-29,1,2018,556256.235616,12,29,0.001798,0.017977


### Statistics on death per day

In [272]:
deaths_normalized

,total,year,population,month,day,death_per_1000
date,,,,,,
0,12,2000,378095.000000,12,31,0.031738
1,8,2001,378156.197260,1,7,0.021155
2,13,2001,378217.394521,1,14,0.034372
3,14,2001,378278.591781,1,21,0.037010
4,7,2001,378339.789041,1,28,0.018502
...,...,...,...,...,...,...
1088,4,2021,574548.671978,11,7,0.006962
1089,10,2021,574666.955144,11,14,0.017401
1090,8,2021,574785.238311,11,21,0.013918


In [69]:
stats.index.to_series()

date
2000-12-31   2000-12-31
2001-01-07   2001-01-07
2001-01-14   2001-01-14
2001-01-21   2001-01-21
2001-01-28   2001-01-28
                ...    
2018-12-02   2018-12-02
2018-12-09   2018-12-09
2018-12-16   2018-12-16
2018-12-23   2018-12-23
2018-12-30   2018-12-30
Name: date, Length: 940, dtype: datetime64[ns]

In [283]:
stats = deaths_normalized.loc['2000':'2018']
stats = stats.groupby(stats.index.to_series().dt.isocalendar().week).agg({
    'death_per_1000':['mean', 'std']}).death_per_1000

In [284]:
stats.index.name= 'weekofyear'
stats

,mean,std
weekofyear,,
1,0.023941,0.010365
2,0.022584,0.008426
3,0.022983,0.006979
4,0.023770,0.008418
5,0.025070,0.007869
6,0.023556,0.004954
7,0.020837,0.007141
8,0.020648,0.010688
9,0.021770,0.008045


In [73]:
deaths_normalized

,total,year,population,death_per_10k,death_per_1000
date,,,,,
2000-12-31,12,2000,378095.000000,0.317381,0.031738
2001-01-07,8,2001,378156.197260,0.211553,0.021155
2001-01-14,13,2001,378217.394521,0.343718,0.034372
2001-01-21,14,2001,378278.591781,0.370098,0.037010
2001-01-28,7,2001,378339.789041,0.185019,0.018502
...,...,...,...,...,...
2021-11-07,4,2021,563109.000000,0.071034,0.007103
2021-11-14,10,2021,563109.000000,0.177586,0.017759
2021-11-21,8,2021,563109.000000,0.142068,0.014207


### Add stats to deaths and calculate z-score

In [285]:
joint = deaths_normalized.set_index(deaths_normalized.index.to_series().dt.isocalendar().week).assign(
    date=deaths_normalized.index).join(stats).sort_values(by='date')

In [286]:
joint

,total,year,population,month,day,death_per_1000,date,mean,std
52,12,2000,378095.000000,12,31,0.031738,2000-12-31,0.024321,0.004978
1,8,2001,378156.197260,1,7,0.021155,2001-01-07,0.023941,0.010365
2,13,2001,378217.394521,1,14,0.034372,2001-01-14,0.022584,0.008426
3,14,2001,378278.591781,1,21,0.037010,2001-01-21,0.022983,0.006979
4,7,2001,378339.789041,1,28,0.018502,2001-01-28,0.023770,0.008418
...,...,...,...,...,...,...,...,...,...
44,4,2021,574548.671978,11,7,0.006962,2021-11-07,0.018821,0.005347
45,10,2021,574666.955144,11,14,0.017401,2021-11-14,0.021973,0.006852
46,8,2021,574785.238311,11,21,0.013918,2021-11-21,0.022450,0.008156
47,5,2021,574903.521478,11,28,0.008697,2021-11-28,0.021370,0.007272


In [287]:
joint['zscore'] = (joint.death_per_1000 - joint['mean']) / joint['std']

In [197]:
joint.columns = 

Index(['report_date', 'total', 'date', 'year', 'population', 'month', 'day',
       'death_per_1000', 'death_per_10k', 'mean', 'std', 'zscore'],
      dtype='object')

In [288]:
charts.Line('date', 'zscore', joint.loc[joint.date.dt.year >= 2019,:], line_width=1, alpha=0.5, color='grey').datetime().opts(width=1000)

{'total': '@total', 'year': '@year', 'population': '@population', 'month': '@month', 'day': '@day', 'death_per_1000': '@death_per_1000', 'date': '@date{%F}', 'mean': '@mean', 'std': '@std', 'zscore': '@zscore'}


Figure(id='5820', ...)

In [141]:
vaxed = pd.read_csv('vax_per_day_40_49.csv')

In [144]:
vaxed.date = pd.to_datetime(vaxed.date)

In [146]:
deaths_and_vaxed = joint.set_index('date').join(vaxed.set_index('date'))

In [167]:
deaths_and_vaxed

,total,date,mean,std,zscore,first_dose,second_dose,third_dose,first_dose_normalized,second_dose_normalized,third_dose_normalized
report_date,,,,,,,,,,,
2000-01-01,2,2000-01-01,1.545455,1.100964,0.412861,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-02,4,2000-01-02,1.545455,1.056827,2.322561,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,3,2000-01-03,1.409091,1.368318,1.162675,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,1,2000-01-04,1.181818,1.097025,-0.165738,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,1,2000-01-05,1.500000,1.439246,-0.347404,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,2,2021-12-01,1.590909,1.053750,0.388224,123.0,109.0,622.0,0.002070,0.001959,0.006058
2021-12-02,2,2021-12-02,1.590909,1.221205,0.334990,118.0,154.0,704.0,0.001986,0.002767,0.006857
2021-12-03,1,2021-12-03,1.500000,1.566008,-0.319283,80.0,72.0,419.0,0.001346,0.001294,0.004081


In [170]:
deaths_and_vaxed['total_normalized'] = deaths_and_vaxed['total'] / deaths_and_vaxed['total'].max()
deaths_and_vaxed['first_dose_normalized'] = deaths_and_vaxed.first_dose / deaths_and_vaxed.first_dose.max()
deaths_and_vaxed['second_dose_normalized'] = deaths_and_vaxed.second_dose / deaths_and_vaxed.second_dose.max()
deaths_and_vaxed['third_dose_normalized'] = deaths_and_vaxed.third_dose / deaths_and_vaxed.third_dose.max()

In [174]:
deaths_and_vaxed

,total,date,mean,std,zscore,first_dose,second_dose,third_dose,first_dose_normalized,second_dose_normalized,third_dose_normalized,total_normalized
report_date,,,,,,,,,,,,
2000-01-01,2,2000-01-01,1.545455,1.100964,0.412861,NaN,NaN,NaN,NaN,NaN,NaN,0.285714
2000-01-02,4,2000-01-02,1.545455,1.056827,2.322561,NaN,NaN,NaN,NaN,NaN,NaN,0.571429
2000-01-03,3,2000-01-03,1.409091,1.368318,1.162675,NaN,NaN,NaN,NaN,NaN,NaN,0.428571
2000-01-04,1,2000-01-04,1.181818,1.097025,-0.165738,NaN,NaN,NaN,NaN,NaN,NaN,0.142857
2000-01-05,1,2000-01-05,1.500000,1.439246,-0.347404,NaN,NaN,NaN,NaN,NaN,NaN,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,2,2021-12-01,1.590909,1.053750,0.388224,123.0,109.0,622.0,0.002070,0.001959,0.006058,0.285714
2021-12-02,2,2021-12-02,1.590909,1.221205,0.334990,118.0,154.0,704.0,0.001986,0.002767,0.006857,0.285714
2021-12-03,1,2021-12-03,1.500000,1.566008,-0.319283,80.0,72.0,419.0,0.001346,0.001294,0.004081,0.142857


In [172]:
deaths_and_vaxed_after_2020 = deaths_and_vaxed.loc['2020-01-01':'2025-01-01']

In [177]:
(
charts.Line('report_date', 'total_normalized', deaths_and_vaxed_after_2020, color='blue').opts(width=800)
    +
charts.Line('report_date', 'first_dose_normalized', deaths_and_vaxed_after_2020, color='red', line_width=3).opts(width=800) 
    +
charts.Line('report_date', 'second_dose_normalized', deaths_and_vaxed_after_2020, color='violet', line_width=3).opts(width=800) 
    +
charts.Line('report_date', 'third_dose_normalized', deaths_and_vaxed_after_2020, color='black', line_width=3).opts(width=800) 
    
)

{'total': '@total', 'date': '@date', 'mean': '@mean', 'std': '@std', 'zscore': '@zscore', 'first_dose': '@first_dose', 'second_dose': '@second_dose', 'third_dose': '@third_dose', 'first_dose_normalized': '@first_dose_normalized', 'second_dose_normalized': '@second_dose_normalized', 'third_dose_normalized': '@third_dose_normalized', 'total_normalized': '@total_normalized'}


Figure(id='28199', ...)

In [180]:
charts.Dots(deaths_and_vaxed_after_2020.third_dose, deaths_and_vaxed_after_2020.total.shift(14))

{'X': '@X', 'Y': '@Y'}


Figure(id='31031', ...)